In [1]:
%pip install pyspark
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 65 kB/s 
     |████████████████████████████████| 199 kB 67.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=abc0e589d5ee1328e43521977ad3e28ba03882bba25595523216a545f2018299
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  I

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-15 16:16:51--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.48MB/s    in 0.2s    

2022-09-15 16:16:52 (5.48 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [5]:
from pyspark.sql.functions import to_date

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1NBG94582SJE2|          5|            0|          0|   N|                Y|
|R273DCA6Y0H9V7|          5|            0|          0|   N|                Y|
| RQVOXO7WUOFK6|          2|            1|          1|   N|                Y|
|R1KWKSF21PO6HO|          5|            0|          0|   N|                Y|
|R38H3UO1J190GI|          5|            1|          1|   N|                Y|
|R3NPIFKLR19NQA|          3|            0|          0|   N|                Y|
|R3MBE6UCH3435E|          3|            8|          8|   N|                N|
|R2E7A4FF0PVY5Q|          5|            0|          1|   N|                Y|
| R3R8JDQ2BF4NM|          5|            0|          2|   N|                Y|
|R1YND4BS823GN5|          1|            0|          0|   N|     

In [8]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

In [9]:
# Filtering to records having total votes >= 20
votes_df = vine_df.filter(col("total_votes") >= 20)
votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R3EM8C9CKA6GPK|          2|           41|        115|   N|                N|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|     

In [10]:
# Filtering with at least 50% helpful votes
helpful_votes_df = votes_df.filter(col("helpful_votes")/col("total_votes") >= 0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|                Y|
|R3JIIPQ8F0P7QS|          4|           22|         23|   N|     

In [11]:
# Filtering to paid vine reviews
vine_reviews_df = helpful_votes_df.filter(col("vine") == "Y")
vine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37L6YZDX0P3J9|          1|           28|         35|   Y|                N|
|R2JT4XRQO2KASL|          5|           38|         41|   Y|                N|
|R10XRAOBFFSTL2|          5|           44|         47|   Y|                N|
| R1X8F3R1K1SF5|          2|           30|         31|   Y|                N|
|R326QV66CKBB43|          4|          127|        139|   Y|                N|
| RGXLFTR31J561|          5|           29|         34|   Y|                N|
|R2ZS3WCJYUDQH0|          3|          101|        115|   Y|                N|
|R31771VBPRDVJS|          4|           75|         83|   Y|                N|
| R8O8KM73GO190|          5|           21|         28|   Y|                N|
| RPU1GURA1NOT0|          5|           33|         35|   Y|     

In [12]:
unpaid_reviews_df = helpful_votes_df.filter(col("vine") == "N")
unpaid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FJ319XA6ZAUQ|          2|           21|         25|   N|                Y|
| R6HRF25HUMIIE|          5|           24|         24|   N|                Y|
|R2CTAK3APOFKZU|          5|           37|         38|   N|                N|
|R1WQZB0CAEPQBU|          2|           85|        128|   N|                N|
| RHZDSA48HVRVK|          5|           49|         86|   N|                N|
|R190J2PDOZ5GVK|          3|           36|         51|   N|                Y|
|R1WZSWWOTN58OP|          5|           73|         77|   N|                Y|
|R29THVJFO35FZA|          5|           48|         50|   N|                Y|
|R3V8FKXIHBLWEL|          5|           91|        108|   N|                Y|
|R3JIIPQ8F0P7QS|          4|           22|         23|   N|     

In [14]:
vine_reviews_count = vine_reviews_df.count()
vine_5_star_reviews_count = vine_reviews_df.filter(col("star_rating") == 5).count()
vine_5_star_review_percent = round(vine_5_star_reviews_count / vine_reviews_count * 100,2)
print(f'''
 Total Vine Reviews: {vine_reviews_count}
 Total 5 Star Reviews: {vine_5_star_reviews_count}
 Percent 5 Star Reviews: {vine_5_star_review_percent}%
     ''')


 Total Vine Reviews: 607
 Total 5 Star Reviews: 257
 Percent 5 Star Reviews: 42.34%
     


In [15]:
unpaid_reviews_count = unpaid_reviews_df.count()
unpaid_5_star_reviews_count = unpaid_reviews_df.filter(col("star_rating") == 5).count()
unpaid_5_star_review_percent = round(unpaid_5_star_reviews_count / unpaid_reviews_count * 100,2)
print(f'''
 Total Unpaid Reviews: {unpaid_reviews_count}
 Total 5 Star Reviews: {unpaid_5_star_reviews_count}
 Percent 5 Star Reviews: {unpaid_5_star_review_percent}%
     ''')


 Total Unpaid Reviews: 50522
 Total 5 Star Reviews: 25220
 Percent 5 Star Reviews: 49.92%
     
